In [1]:
import pandas as pd
import numpy as np
import itertools
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
def setup_mpl():
    mpl.rc('font', size=15)
    mpl.rcParams['xtick.major.pad']='12'
    mpl.rcParams['ytick.major.pad']='12'
    mpl.rcParams['lines.linewidth'] = 1
    mpl.rcParams['xtick.major.width'] = 2.5
    mpl.rcParams['ytick.major.width'] = 2.5
    mpl.rcParams['xtick.minor.width'] = 1.5
    mpl.rcParams['ytick.minor.width'] = 1.5
    mpl.rcParams['xtick.major.size'] = 6
    mpl.rcParams['ytick.major.size'] = 6
    mpl.rcParams['xtick.minor.size'] = 3
    mpl.rcParams['ytick.minor.size'] = 3
    mpl.rcParams['axes.spines.right'] = True
    mpl.rcParams['axes.spines.top'] = True
    mpl.rcParams['xtick.minor.size'] = 1
    mpl.rcParams['lines.linewidth']=2
setup_mpl()

# Compute normalized&non-normalized correlations

In [2]:
# Correlation functions
def compute_spearman_correlation(x):
    pair1, pair2, connection_time = x.copy()
    #Compute rolling correlation using pandas (simply look at the rank instead of the values)
    res = df_measure[pair1].rank().rolling(WINDOW_SIZE, min_periods=MIN_WINDOW_SIZE).corr(pairwise=True, other = df_measure[pair2].rank())
    #shift (by window size) and substract connection time
    res.index = (res.index-connection_time)# - datetime.timedelta(int(WINDOW_SIZE))
    return res

def compute_spearman_correlation_at_t(x):
    pair1, pair2 = x.copy()
    #Compute rolling correlation using pandas (simply look at the rank instead of the values)
    res = df_measure[pair1].rank().rolling(WINDOW_SIZE, min_periods=MIN_WINDOW_SIZE).corr(pairwise=True, other = df_measure[pair2].rank())
    return res

def compute_normalized_spearman_correlation(x):
    pair1, pair2, connection_time = x.copy()
    #Compute rolling correlation using pandas (simply look at the rank instead of the values)
    res = df_measure[pair1].rank().rolling(WINDOW_SIZE, min_periods=MIN_WINDOW_SIZE).corr(pairwise=True, other = df_measure[pair2].rank())
    res_norm = ((res-average_corr)/std_corr).dropna()

    res_norm.index = (res_norm.index - connection_time)
    return res_norm 

# Iterate over different window sizes

In [ ]:
##### 
WINDOW_SIZE_LIST = np.array([60,80,100,120,140,160,180])
MIN_PERCENTAGE_OF_COLUMNS = 0.05
DAY_LIM = 600
TRANSITION_LENGTH = 150

##################### LOADING DATA ###########################
# LOAD TIMESERIES "measure"
start_date = "01-01-2014" # 
end_date = "30-10-2019"   # 


for measure_serie in ['retur']:#,'price','mrcap','mrret','volum','volumret','volat','volatret']:
    # Select the desired time period from market data
    df_measure = pd.read_pickle('export/df_{}.pkl'.format(measure_serie))
    tmp = df_measure[(df_measure.index<=pd.to_datetime(end_date,format='%d-%m-%Y'))&(df_measure.index>=pd.to_datetime(start_date,format='%d-%m-%Y'))].drop('year',axis=1)
    df_measure = tmp[tmp.isna().all()[tmp.isna().all()<1].index]

    #  LOAD CONNECTED COUPLES 
    edges_file_dir = 'conn_couples'         # input couples (random or real couples)
    df_edges = pd.read_pickle(edges_file_dir+'_safe.pkl')
    df_edges = df_edges[df_edges.coll_begins>df_measure.first_valid_index()]
    tmp_list = df_edges.org1.append(df_edges.org2).drop_duplicates()
    df_list = list(tmp_list[tmp_list.isin(list(df_measure.columns))])
    df_edges = df_edges[(df_edges.coll_begins>=pd.to_datetime(start_date,format='%d-%m-%Y'))&(df_edges.coll_begins<=pd.to_datetime(end_date,format='%d-%m-%Y'))]
    df_edges = df_edges[(df_edges.org1.isin(df_list))&(df_edges.org2.isin(df_list))].reset_index(drop=True)
    df_edges.to_pickle(edges_file_dir+'.pkl')
    
    #  LOAD RANDOM T+A COUPLES (RTA)
    edges_file_dir = 'rand_conn_time_and_age_couples'         # input couples (random or real couples)
    rRTA_edges = pd.read_pickle(edges_file_dir+'_safe.pkl')
    rRTA_edges = rRTA_edges[rRTA_edges.coll_begins>df_measure.first_valid_index()]
    tmp_list = rRTA_edges.org1.append(rRTA_edges.org2).drop_duplicates()
    df_list = list(tmp_list[tmp_list.isin(list(df_measure.columns))])
    rRTA_edges = rRTA_edges[(rRTA_edges.coll_begins>=pd.to_datetime(start_date,format='%d-%m-%Y'))&(rRTA_edges.coll_begins<=pd.to_datetime(end_date,format='%d-%m-%Y'))]
    rRTA_edges = rRTA_edges[(rRTA_edges.org1.isin(df_list))&(rRTA_edges.org2.isin(df_list))].reset_index(drop=True)
    rRTA_edges = rRTA_edges.drop_duplicates(['org1','org2','coll_begins']).reset_index(drop=True)
    rRTA_edges = rRTA_edges.merge(rRTA_edges.rename(columns={'org1':'org2','org2':'org1'}),on=['org1','org2','coll_begins'], indicator=True,how='left').query('_merge=="left_only"').reset_index(drop=True)
    rRTA_edges.to_pickle(edges_file_dir+'.pkl')
    
    #  LOAD RANDOM T+A COUPLES (RT)
    edges_file_dir = 'rand_conn_time_couples'
    rRT_edges = pd.read_pickle(edges_file_dir+'_safe.pkl')
    rRT_edges = rRT_edges[rRT_edges.coll_begins>df_measure.first_valid_index()]
    tmp_list = rRT_edges.org1.append(rRT_edges.org2).drop_duplicates()
    df_list = list(tmp_list[tmp_list.isin(list(df_measure.columns))])
    rRT_edges = rRT_edges[(rRT_edges.coll_begins>=pd.to_datetime(start_date,format='%d-%m-%Y'))&(rRT_edges.coll_begins<=pd.to_datetime(end_date,format='%d-%m-%Y'))]
    rRT_edges = rRT_edges[(rRT_edges.org1.isin(df_list))&(rRT_edges.org2.isin(df_list))].reset_index(drop=True)
    rRT_edges = rRT_edges.drop_duplicates(['org1','org2','coll_begins']).reset_index(drop=True)
    rRT_edges = rRT_edges.merge(rRT_edges.rename(columns={'org1':'org2','org2':'org1'}),on=['org1','org2','coll_begins'], indicator=True,how='left').query('_merge=="left_only"').reset_index(drop=True)
    rRT_edges.to_pickle(edges_file_dir+'.pkl')

    #  LOAD ONE RANDOM T COUPLES (ORT)
    edges_file_dir = 'rand_conn_time_couples'
    rORT_edges = pd.read_pickle(edges_file_dir+'_safe.pkl')
    rORT1 = rORT_edges.copy()
    rORT1['org1'] = rORT1.original_org1
    rORT2 = rORT_edges.copy()
    rORT2['org2'] = rORT2.original_org2
    rORT_edges = rORT1.append(rORT2).reset_index(drop=True)
    rORT_edges = rORT_edges[rORT_edges.coll_begins>df_measure.first_valid_index()]
    tmp_list = rORT_edges.org1.append(rORT_edges.org2).drop_duplicates()
    df_list = list(tmp_list[tmp_list.isin(list(df_measure.columns))])
    rORT_edges = rORT_edges[(rORT_edges.coll_begins>=pd.to_datetime(start_date,format='%d-%m-%Y'))&(rORT_edges.coll_begins<=pd.to_datetime(end_date,format='%d-%m-%Y'))]
    rORT_edges = rORT_edges[(rORT_edges.org1.isin(df_list))&(rORT_edges.org2.isin(df_list))].reset_index(drop=True)
    rORT_edges = rORT_edges.drop_duplicates(['org1','org2','coll_begins']).reset_index(drop=True)
    rORT_edges = rORT_edges.merge(rORT_edges.rename(columns={'org1':'org2','org2':'org1'}),on=['org1','org2','coll_begins'], indicator=True,how='left').query('_merge=="left_only"').reset_index(drop=True)
    rORT_edges = rORT_edges.sample(len(rRT_edges),replace=True).reset_index(drop=True)

    #  LOAD ONE RANDOM T+A COUPLES (ORTA)
    edges_file_dir = 'rand_conn_time_and_age_couples'
    rORTA_edges = pd.read_pickle(edges_file_dir+'_safe.pkl')
    rORTA1 = rORTA_edges.copy()
    rORTA1['org1'] = rORTA1.original_org1
    rORTA2 = rORTA_edges.copy()
    rORTA2['org2'] = rORTA2.original_org2
    rORTA_edges = rORTA1.append(rORTA2).reset_index(drop=True)
    rORTA_edges = rORTA_edges[rORTA_edges.coll_begins>df_measure.first_valid_index()]
    tmp_list = rORTA_edges.org1.append(rORTA_edges.org2).drop_duplicates()
    df_list = list(tmp_list[tmp_list.isin(list(df_measure.columns))])
    rORTA_edges = rORTA_edges[(rORTA_edges.coll_begins>=pd.to_datetime(start_date,format='%d-%m-%Y'))&(rORTA_edges.coll_begins<=pd.to_datetime(end_date,format='%d-%m-%Y'))]
    rORTA_edges = rORTA_edges[(rORTA_edges.org1.isin(df_list))&(rORTA_edges.org2.isin(df_list))].reset_index(drop=True)
    rORTA_edges = rORTA_edges.drop_duplicates(['org1','org2','coll_begins']).reset_index(drop=True)
    rORTA_edges = rORTA_edges.merge(rORTA_edges.rename(columns={'org1':'org2','org2':'org1'}),on=['org1','org2','coll_begins'], indicator=True,how='left').query('_merge=="left_only"').reset_index(drop=True)
    ##################### END LOADING ###########################

    # Connected Couples
    pairs = df_edges[['org1','org2','coll_begins']]
    pairsRTA = rRTA_edges[['org1','org2','coll_begins']]
    pairsRT = rRT_edges[['org1','org2','coll_begins']]
    pairsORT = rORT_edges[['org1','org2','coll_begins']]
    pairsORTA = rORTA_edges[['org1','org2','coll_begins']]

    # Remove connected couples from random couples
    pairsRT = pairsRT.merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)
    pairsRT = pairsRT.rename(columns={'org1':'org2','org2':'org1'}).merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)

    pairsRTA = pairsRTA.merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)
    pairsRTA = pairsRTA.rename(columns={'org1':'org2','org2':'org1'}).merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)

    pairsORT = pairsORT.merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)
    pairsORT = pairsORT.rename(columns={'org1':'org2','org2':'org1'}).merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)
    pairsORTA = pairsORTA.merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)
    pairsORTA = pairsORTA.rename(columns={'org1':'org2','org2':'org1'}).merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)

    # Ecology representatives Random Couples
    cryptos = df_measure.dropna(how='all',axis=1).columns
    crypto_pairs = pd.DataFrame(itertools.combinations(cryptos,2))
    # ## Remove connected couples from crypto_pairs
    crypto_pairs = crypto_pairs.rename(columns={0:'org1',1:'org2'}).merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)
    crypto_pairs = crypto_pairs.rename(columns={0:'org2',1:'org1'}).merge(pairs[['org1','org2']],on=['org1','org2'], indicator=True,how='left').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)

    print('Connected couples: {}'.format(len(pairs)))
    print('RT couples: {}'.format(len(pairsRT)))
    print('RTA couples: {}'.format(len(pairsRTA)))
    print('ORT couples: {}'.format(len(pairsORT)))
    print('ORTA couples: {}'.format(len(pairsORTA)))

    for WINDOW_SIZE in WINDOW_SIZE_LIST:
        MIN_WINDOW_SIZE = int(0.95*WINDOW_SIZE)
         ## SPEARMAN CORRELATION
        r = pairs.apply(compute_spearman_correlation,axis = 1).T
        r.replace(np.inf,np.NaN,inplace=True)
        r.replace(-np.inf,np.NaN,inplace=True)
        r = r.dropna(axis=1,how='all')
        # remove indexes with low statistics
        threshold = r.count(axis=1)/len(r.columns)
        threshold = threshold[(threshold>MIN_PERCENTAGE_OF_COLUMNS)&(threshold.index>=f'-{DAY_LIM} days')&(threshold.index<=f'{DAY_LIM+TRANSITION_LENGTH} days')]
        r = r.loc[threshold.index]

        rmean = r.mean(axis = 1)
        rstd = r.sem(axis = 1)

    ## RTA couples correlation
        rRTA = pairsRTA.apply(compute_spearman_correlation,axis = 1).T
        rRTA.replace(np.inf,np.NaN,inplace=True)
        rRTA.replace(-np.inf,np.NaN,inplace=True)
        rRTA = rRTA.loc[threshold.index]
        rmeanRTA = rRTA.mean(axis = 1)
        rstdRTA = rRTA.sem(axis = 1)
    ## ORTA couples correlation
        rORTA = pairsORTA.apply(compute_spearman_correlation,axis = 1).T
        rORTA.replace(np.inf,np.NaN,inplace=True)
        rORTA.replace(-np.inf,np.NaN,inplace=True)
        rORTA = rORTA.loc[threshold.index]
        rmeanORTA = rORTA.mean(axis = 1)
        rstdORTA = rORTA.sem(axis = 1)

    ## RT couples correlation
        rRT = pairsRT.apply(compute_spearman_correlation,axis = 1).T
        rRT.replace(np.inf,np.NaN,inplace=True)
        rRT.replace(-np.inf,np.NaN,inplace=True)
        rRT = rRT.loc[threshold.index]
        rmeanRT = rRT.mean(axis = 1)
        rstdRT = rRT.sem(axis = 1)
    ## ORT couples correlation
        rORT = pairsORT.apply(compute_spearman_correlation,axis = 1).T
        rORT.replace(np.inf,np.NaN,inplace=True)
        rORT.replace(-np.inf,np.NaN,inplace=True)
        rORT = rORT.loc[threshold.index]
        rmeanORT = rORT.mean(axis = 1)
        rstdORT = rORT.sem(axis = 1)

    ### End of correlation ###        
    # ##### NORMALIZED CORRELATION #####
        #TAKE ONLY A RANDOM SAMPLE OF CRYPTOS AND COMPUTE THE DAILY CORRELATION
        t = time.time()
        rt = crypto_pairs.apply(compute_spearman_correlation_at_t,axis = 1)
        print(time.time() -t)
        rt.replace(np.inf,np.NaN,inplace=True)
        rt.replace(-np.inf,np.NaN,inplace=True)
    ### comment the following 2 lines if you want to remove non-active cryptos from normalization 
    ### (uncomment lines in "compute_normalized_spearman_correlation")
        average_corr = rt.mean()
        std_corr = rt.std()

        #Compute the normalized daily correlation
        rp = pairs.apply(compute_normalized_spearman_correlation,axis = 1).T
        rp.replace(np.inf,np.NaN,inplace=True)
        rp.replace(-np.inf,np.NaN,inplace=True)
        rp = rp.dropna(axis=1,how='all')
        # remove indexes with low statistics
        thresholdp = rp.count(axis=1)/len(rp.columns)
        thresholdp = thresholdp[(thresholdp>MIN_PERCENTAGE_OF_COLUMNS)&(thresholdp.index>=f'-{DAY_LIM} days')&(thresholdp.index<=f'{DAY_LIM+TRANSITION_LENGTH} days')]
        rp = rp.loc[thresholdp.index]
        rpmean = rp.mean(axis = 1)
        rpstd = rp.sem(axis = 1)

    ## RTA stdandardized couples correlation
        rpRTA = pairsRTA.apply(compute_normalized_spearman_correlation,axis = 1).T
        rpRTA.replace(np.inf,np.NaN,inplace=True)
        rpRTA.replace(-np.inf,np.NaN,inplace=True)
        rpRTA = rpRTA.loc[thresholdp.index]
        rpmeanRTA = rpRTA.mean(axis = 1)
        rpstdRTA = rpRTA.sem(axis = 1)
    ## ORTA stdandardized couples correlation
        rpORTA = pairsORTA.apply(compute_normalized_spearman_correlation,axis = 1).T
        rpORTA.replace(np.inf,np.NaN,inplace=True)
        rpORTA.replace(-np.inf,np.NaN,inplace=True)
        rpORTA = rpORTA.loc[thresholdp.index]
        rpmeanORTA = rpORTA.mean(axis = 1)
        rpstdORTA = rpORTA.sem(axis = 1)

    ## RT stdandardized couples correlation
        rpRT = pairsRT.apply(compute_normalized_spearman_correlation,axis = 1).T
        rpRT.replace(np.inf,np.NaN,inplace=True)
        rpRT.replace(-np.inf,np.NaN,inplace=True)
        rpRT = rpRT.loc[thresholdp.index]
        rpmeanRT = rpRT.mean(axis = 1)
        rpstdRT = rpRT.sem(axis = 1)
    ## ORT couples correlation
        rpORT = pairsORT.apply(compute_normalized_spearman_correlation,axis = 1).T
        rpORT.replace(np.inf,np.NaN,inplace=True)
        rpORT.replace(-np.inf,np.NaN,inplace=True)
        rpORT = rpORT.loc[threshold.index]
        rpmeanORT = rpORT.mean(axis = 1)
        rpstdORT = rpORT.sem(axis = 1)


        r.to_pickle(f'export/robustness/r_{measure_serie}_win{WINDOW_SIZE}.pkl')
        rRTA.to_pickle(f'export/robustness/rRTA_{measure_serie}_win{WINDOW_SIZE}.pkl')
        rORTA.to_pickle(f'export/robustness/rORTA_{measure_serie}_win{WINDOW_SIZE}.pkl')
        rRT.to_pickle(f'export/robustness/rRT_{measure_serie}_win{WINDOW_SIZE}.pkl')
        rORT.to_pickle(f'export/robustness/rORT_{measure_serie}_win{WINDOW_SIZE}.pkl')

        rp.to_pickle(f'export/robustness/rp_{measure_serie}_win{WINDOW_SIZE}.pkl')
        rpRTA.to_pickle(f'export/robustness/rpRTA_{measure_serie}_win{WINDOW_SIZE}.pkl')
        rpORTA.to_pickle(f'export/robustness/rpORTA_{measure_serie}_win{WINDOW_SIZE}.pkl')
        rpRT.to_pickle(f'export/robustness/rpRT_{measure_serie}_win{WINDOW_SIZE}.pkl')
        rpORT.to_pickle(f'export/robustness/rpORT_{measure_serie}_win{WINDOW_SIZE}.pkl')


Connected couples: 204
RT couples: 96920
RTA couples: 6296
ORT couples: 94393
ORTA couples: 1714


In [ ]:
### Bootstrapping 
sample_number = 10000
measures = ['retur']#,'mrcap','price','mrret','volum','volat','volumret']
windows = [60,80,100,120,140,160,180]
def boot_sem(x):
    days = x.name
    x_val = x.dropna()
    tmp_mean = np.zeros(sample_number)
    for i in range(sample_number):
        tmp_sample = x_val.sample(size_serie.loc[days],replace=True)
        tmp_mean[i] = tmp_sample.mean()
    return [np.std(tmp_mean),np.mean(tmp_mean)]

def boot_seM(x):
    days = x.name
    x_val = x.dropna()
    tmp_mean = np.zeros(sample_number)
    for i in range(sample_number):
        tmp_sample = x_val.sample(size_serie.loc[days],replace=True)
        tmp_mean[i] = tmp_sample.median()
    return [np.std(tmp_mean),np.mean(tmp_mean),np.median(tmp_mean),np.quantile(tmp_mean,0.025),np.quantile(tmp_mean,0.975)]

### ROBUSTNESS TEST -> Median instead of mean (only for one window) 
sample_number = 10000
measure = 'retur'
window = 120

df_tmp = pd.read_pickle(f'export/robustness/rp_{measure}_win{window}.pkl')
df_tmp_time_rand = pd.read_pickle(f'export/robustness/rpRT_{measure}_win{window}.pkl')

dfr = pd.DataFrame(index=df_tmp.index)
dforta = pd.DataFrame(index=df_tmp.index)
dfrta = pd.DataFrame(index=df_tmp.index)
dfrt = pd.DataFrame(index=df_tmp.index)
dfr['avg'] = df_tmp.mean(axis=1)
dfr['sem'] = df_tmp.sem(axis=1)
dfr['std'] = df_tmp.std(axis=1)
size_serie = df_tmp.count(axis=1)
dfr[['sem_N','avg_N']] = df_tmp.apply(lambda x: boot_sem(x),axis=1,result_type='expand')
dfr[['sem_M','avg_M','med_M','left_quant_M','right_quant_M']] = df_tmp.apply(lambda x: boot_seM(x),axis=1,result_type='expand')
dfr['active_cryptos'] = df_tmp.count(axis=1)

dfrt['avg'] = df_tmp_time_rand.mean(axis=1)
dfrt['sem'] = df_tmp_time_rand.sem(axis=1)
dfrt['std'] = df_tmp_time_rand.std(axis=1)
size_serie = df_tmp.count(axis=1)
dfrt[['sem_N','avg_N']] = df_tmp_time_rand.apply(lambda x: boot_sem(x),axis=1,result_type='expand')
dfrt[['sem_M','avg_M','med_M','left_quant_M','right_quant_M']] = df_tmp_time_rand.apply(lambda x: boot_seM(x),axis=1,result_type='expand')
dfrt['active_cryptos'] = df_tmp_time_rand.count(axis=1)

dfr.to_pickle(f'export/sCC_summary_{measure}_{sample_number}_win{window}_M.pkl')
dfrt.to_pickle(f'export/sRT_summary_{measure}_{sample_number}_win{window}_M.pkl')


In [ ]:
### For long run get notified when computations finish
# from scriptUltraNotifier_bot import *
# load_logger('Notebook 4 robustness first ended!)